In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import gc

In [ ]:
# filtered_ratings_df = pd.read_csv("Filtered_Ratings.csv",dtype={"ISBN": str}, low_memory=False)
# fr0_df = filtered_ratings_df[filtered_ratings_df['Book-Rating'] != 0]
# fr0_df
# fr0_df.to_csv("Filtered_Ratings_NoZero.csv", index=False)

* Latent Factor Models (SVD : R = Q*Pt)
* unrated books에 대해서는 0이 아닌 NaN으로 유지
* Sparse SVD를 통해 누락된 값(NaN) 무시하고 데이터가 있는 값들만 사용하여 분해를 수행

In [ ]:
# fr0_df
# row_cnt = fr0_df["User-ID"].count()
# row_cnt
# #R_matrix = fr0_df.pivot(index='ISBN', columns='User-ID', values='Book-Rating')

In [ ]:
# rating_nozero_df = pd.read_csv("Filtered_Ratings_NoZero.csv")
# R_matrix = rating_nozero_df.pivot(index='ISBN', columns='User-ID', values='Book-Rating')

In [ ]:
# 고유한 user-id 개수
# rating_nozero_df = pd.read_csv("Filtered_Ratings_NoZero.csv")
# userid_cnt = rating_nozero_df["User-ID"].nunique()
# isbn_cnt = rating_nozero_df["ISBN"].nunique()
# row_cnt = rating_nozero_df["User-ID"].count()
# print("unique User-ID : ", userid_cnt)
# print("unique ISBN : ", isbn_cnt)
# print("row count : ", row_cnt)

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col
# from pyspark.mllib.linalg.distributed import RowMatrix

# # spark 세션 시작
# spark = SparkSession.builder.appName("ratingMatrix").config("spark.sql.pivotMaxValues", 10000).config("spark.driver.memory", "16g").config("spark.executor.memory", "16g").getOrCreate()
# #csv 파일을 Spark DF로 로드
# df = spark.read.csv("Filtered_Ratings_NoZero.csv", header=True, inferSchema=True)

# # 처음 10만개 데이터만 선택
# limited_df = df.limit(10000)

# #행렬 변환
# R_matrix = limited_df.groupBy("ISBN").pivot("User-ID").sum("Book-Rating")

# #MLLib 사용 - 대규모 dataset이기 때문에 분산환경에서 SVD 진행해야함

# #R.matrix -> RDD로 변환
# rows = R_matrix.rdd.map(lambda row: Vectors.dense(row[1:]))

# # 행렬 객체 생성
# mat = RowMatrix(rows)

# k = 50
# # SVD 수행
# svd = mat.computeSVD(k, computeU=True)
# U = svd.U       # U 행렬
# s = svd.s       # Sigma 행렬의 대각 원소
# V = svd.V       # V 행렬

In [2]:
df = pd.read_csv("Filtered_Ratings_NoZero.csv")

print(df.head())

   User-ID        ISBN  Book-Rating
0   276726  0155061224            5
1   276729  052165615X            3
2   276729  0521795028            6
3   276744  038550120X            7
4   276747  0060517794            9


In [20]:
# User-ID별로 등장 횟수를 세어 Series로 저장
user_counts = df['User-ID'].value_counts()

# User-ID가 100번 이상 등장한 사용자만 남기기
filtered_df = df[df['User-ID'].isin(user_counts[user_counts >= 100].index)]

# 결과 출력
total_rows = len(filtered_df)
print(filtered_df.head())
print("행의 개수 : ", total_rows)

     User-ID        ISBN  Book-Rating
529   277427  002542730X           10
530   277427  003008685X            8
531   277427  0060006641           10
532   277427  0060542128            7
533   277427  0061009059            9
행의 개수 :  103271


In [21]:
sample = 80000
df_limited = filtered_df.head(sample)

R_matrix = df_limited.pivot(index="ISBN", columns="User-ID", values="Book-Rating").fillna(0).astype("float32")
# print(R_matrix.head())

In [22]:
num_rows = R_matrix.shape[0]  # 행의 개수
num_cols = R_matrix.shape[1]  # 열의 개수

print(f"행의 개수: {num_rows}")
print(f"열의 개수: {num_cols}")

행의 개수: 55929
열의 개수: 324


In [6]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds

In [7]:
# 행렬을 희소 행렬 형태로 변환
A = csc_matrix(R_matrix.values)

# SVD 수행, k는 추출할 특성의 수
u, s, vt = svds(A, k=100)

In [ ]:
print("U 행렬:")
print(u)

In [ ]:
print("S 행렬 : ")
print(s)

In [ ]:
print("Vt 행렬 : ")
print(vt)

In [8]:
# 데이터 복원
# np.diag() : 특이값들을 대각 행렬로 나타내고, 상위 k개의 특이값만 남기고 나머지는 0으로 만듦
reconstructed_data = np.dot(np.dot(u, np.diag(s)), vt)
print("복원된 데이터:")
print(reconstructed_data)

복원된 데이터:
[[ 3.11993454e-02 -9.59490426e-03 -7.47691840e-04 ...  2.53759115e-03
   6.19416684e-02  9.04379599e-03]
 [ 2.26132572e-04 -7.70963728e-04 -9.33469273e-03 ... -1.08570477e-03
  -2.07405724e-02  1.09242126e-02]
 [ 1.50755048e-04 -5.13976440e-04 -6.22312445e-03 ... -7.23795907e-04
  -1.38270445e-02  7.28280842e-03]
 ...
 [ 2.55859061e-03 -2.98895920e-05  9.55221418e-04 ... -1.75514084e-03
   6.47462672e-03  1.57948249e-04]
 [ 2.72663776e-02  1.38000632e-02 -6.82903677e-02 ...  3.26084159e-02
   6.47109672e-02 -2.87385727e-03]
 [ 8.72747228e-03 -1.86603330e-02 -5.06761717e-03 ... -4.66174679e-04
   5.39789684e-02  9.46346391e-03]]


In [7]:
# error = np.linalg.norm(R_matrix - reconstructed_data)
# print("Error : ", error)

Error :  1899.1826


In [9]:
# R_matrix에서 rating이 0이 아닌 (userid, isbn, rating) 50개 뽑아냄
non_zero_ratings = []
# Iterate through the R_matrix
for user_idx, user_row in enumerate(R_matrix.index):
    for book_idx, book_isbn in enumerate(R_matrix.columns):
        rating = R_matrix.iloc[user_idx, book_idx]
        if rating != 0:
            non_zero_ratings.append((user_idx, book_idx, rating))
            


# Display the non-zero ratings
# for user_idx, book_idx, rating in non_zero_ratings:
#     user_id = R_matrix.index[user_idx]
#     book_isbn = R_matrix.columns[book_idx]
#     print("User-ID:", user_id)
#     print("ISBN:", book_isbn)
#     print("Rating:", rating)
#     print("-----------------------------")

KeyboardInterrupt: 

In [35]:
# 실제 값과 예측값 비교하고 error 평균 출력 (error = 기존 값 - 예측 값)
def compare_ratings(non_zero_ratings, reconstructed_data):
    error_sum = 0
    for user_idx, book_idx, rating in non_zero_ratings:
        user_id = R_matrix.index[user_idx]
        book_isbn = R_matrix.columns[book_idx]
        original_rating = rating
        estimated_rating = reconstructed_data[user_idx, book_idx]
        error_sum += abs(original_rating - estimated_rating)
        print("User-ID:", user_id)
        print("ISBN:", book_isbn)
        print("original_rating:", original_rating)
        print("estimated_rating:", estimated_rating)
        print("----------------------------------------------------------")
    return (error_sum/len(non_zero_ratings))

In [ ]:
# 기존 matrix에서 상위 8만개 행에 대해서 svd 진행
error = compare_ratings(non_zero_ratings, reconstructed_data)
print("평균 에러 값: ", error)

In [43]:
# 평가 100번 이상 한 유저(10만개 행) 중 8만개 행 뽑아서 진행
# k = 50
error = compare_ratings(non_zero_ratings, reconstructed_data)
print("평균 에러 값: ", error)

User-ID: 0000913154
ISBN: 171118
original_rating: 8.0
estimated_rating: 7.964249
----------------------------------------------------------
User-ID: 0001046438
ISBN: 23902
original_rating: 9.0
estimated_rating: 8.991164
----------------------------------------------------------
User-ID: 000104687X
ISBN: 23902
original_rating: 6.0
estimated_rating: 5.9941087
----------------------------------------------------------
User-ID: 0001047213
ISBN: 23902
original_rating: 9.0
estimated_rating: 8.991164
----------------------------------------------------------
User-ID: 0001047973
ISBN: 23902
original_rating: 9.0
estimated_rating: 8.991164
----------------------------------------------------------
User-ID: 000104799X
ISBN: 166596
original_rating: 8.0
estimated_rating: 0.012464575
----------------------------------------------------------
User-ID: 0001048082
ISBN: 23902
original_rating: 8.0
estimated_rating: 7.9955406
----------------------------------------------------------
User-ID: 0001048082


In [46]:
# 평가 100번 이상 한 유저(10만개 행) 중 8만개 행 뽑아서 진행
# k = 75
error = compare_ratings(non_zero_ratings, reconstructed_data)
print("평균 에러 값: ", error)

User-ID: 0000913154
ISBN: 171118
original_rating: 8.0
estimated_rating: 7.971637
----------------------------------------------------------
User-ID: 0001046438
ISBN: 23902
original_rating: 9.0
estimated_rating: 8.992366
----------------------------------------------------------
User-ID: 000104687X
ISBN: 23902
original_rating: 6.0
estimated_rating: 5.994912
----------------------------------------------------------
User-ID: 0001047213
ISBN: 23902
original_rating: 9.0
estimated_rating: 8.992366
----------------------------------------------------------
User-ID: 0001047973
ISBN: 23902
original_rating: 9.0
estimated_rating: 8.992366
----------------------------------------------------------
User-ID: 000104799X
ISBN: 166596
original_rating: 8.0
estimated_rating: 0.025527962
----------------------------------------------------------
User-ID: 0001048082
ISBN: 23902
original_rating: 8.0
estimated_rating: 7.996132
----------------------------------------------------------
User-ID: 0001048082
IS

In [61]:
# 평가 100번 이상 한 유저(10만개 행) 샘플 안뽑고 그대로 진행
# k = 75
reconstruction_error = np.linalg.norm(R_matrix - reconstructed_data)
print("평균 에러 값: ", reconstruction_error)

평균 에러 값:  1842.0021


In [66]:
# 평가 100번 이상 한 유저(10만개 행) 중 8만개 행 뽑아서 진행
# k = 75
reconstruction_error = np.linalg.norm(R_matrix - reconstructed_data)
print("평균 에러 값: ", reconstruction_error)

평균 에러 값:  1467.6725


In [69]:
# 평가 100번 이상 한 유저(10만개 행) 중 8만개 행 뽑아서 진행
# k = 50
reconstruction_error = np.linalg.norm(R_matrix - reconstructed_data)
print("평균 에러 값: ", reconstruction_error)

평균 에러 값:  1599.739


In [74]:
# 그냥 8만개 행 뽑아서 진행
# k = 50
reconstruction_error = np.linalg.norm(R_matrix - reconstructed_data)
print("평균 에러 값: ", reconstruction_error)

평균 에러 값:  1899.1825


In [80]:
#평가 200번 이상 한 유저 전부(58174 행)
# k = 50
reconstruction_error = np.linalg.norm(R_matrix - reconstructed_data)
print("에러 값: ", reconstruction_error)

에러 값:  989.6968


In [83]:
#평가 200번 이상 한 유저 전부(58174 행)
# k = 100
reconstruction_error = np.linalg.norm(R_matrix - reconstructed_data)
print("에러 값: ", reconstruction_error)

에러 값:  421.18137


In [10]:
# 평가 100번 8만개 k = 100
reconstruction_error = np.linalg.norm(R_matrix - reconstructed_data)
print("에러 값: ", reconstruction_error)

에러 값:  1347.1295
